# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into Deep Neural Networks for NLP!

## Objectives

You will be able to:

* Demonstrate a basic understanding of the architecture of the Word2Vec model
* Demonstrate an understanding of the various tunable parameters of Word2Vec such as vector size and window size

## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec Model. Then, you'll move onto building Neural Networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [3]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a github repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json` function from pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've loaded in the data, inspect the head of the DataFrame to see what your data looks like. 

In [7]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data--`headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `combined_text` that consists of the data from `df.headline` plus a space character (`' '`) plus the data from `df.short_description`.
* Use the `combined_text` column's `map()` function and pass in `word_tokenize`. Store the result returned in `data`.

In [8]:
df['combined_text'] = df.headline + ' ' +  df.short_description
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [9]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from gensim below. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1`.
    * The number of threads to use during training, `workers=4`

In [10]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call `model.train()` and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count`. 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [11]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55565567, 67337030)

Great! you now have a fully trained model! The word vectors themselves are stored inside of a `Word2VecKeyedVectors` instance, which is stored inside of `model.wv`. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [12]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [13]:
wv.most_similar('Texas')

/home/a00005987/.local/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Arkansas', 0.812136173248291),
 ('Pennsylvania', 0.8074214458465576),
 ('Connecticut', 0.8037616014480591),
 ('Maryland', 0.8006600141525269),
 ('Ohio', 0.7910088300704956),
 ('Louisiana', 0.7909313440322876),
 ('California', 0.7890207767486572),
 ('Illinois', 0.7874826192855835),
 ('Massachusetts', 0.7868660092353821),
 ('Oregon', 0.7839927673339844)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `most_similar()` function's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [14]:
wv.most_similar(negative='Texas')

[('PROVINCETOWN', 0.3897869288921356),
 ('maitre', 0.3886113166809082),
 ('Parent/Grandparent', 0.36315420269966125),
 ('Revolve', 0.35937947034835815),
 ('Frivolous', 0.3566362261772156),
 ('writer-producer', 0.3550916016101837),
 ("'Trash", 0.3470100164413452),
 ('Lunatics', 0.34453028440475464),
 ('End-of-School', 0.3422248661518097),
 ("'Banned", 0.34161773324012756)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [15]:
wv['Texas']

array([ 5.09689271e-01,  1.31138086e-01, -4.45735395e-01,  1.61839676e+00,
        1.10756612e+00, -1.53997958e+00,  1.96665573e+00, -1.13683963e+00,
       -1.96443152e+00, -1.26648515e-01, -2.00377083e+00,  4.02224451e-01,
       -1.50361192e+00,  2.71085411e-01,  2.94082856e+00,  1.02821636e+00,
       -1.86742806e+00, -3.74221116e-01, -6.22936666e-01,  5.34099698e-01,
       -9.30109143e-01,  8.80933404e-01,  1.97472870e-01,  1.05185226e-01,
       -9.18855846e-01, -1.36418343e-01,  1.24205279e+00, -2.73177242e+00,
        9.93159771e-01, -3.68345308e+00, -4.15647894e-01, -3.59486222e-01,
        2.04595804e+00,  1.71551800e+00, -4.62892473e-01, -1.86722457e-01,
       -1.17527223e+00, -1.67770183e+00, -2.21513256e-01, -1.29731643e+00,
        1.20213532e+00, -1.40219927e+00,  3.36913198e-01,  1.99511945e+00,
        1.62821352e+00,  1.79223865e-02,  5.56844413e-01, -8.69604349e-01,
        1.81302881e+00,  1.06073391e+00,  2.02700853e-01, -1.49748671e+00,
        1.70774865e+00, -

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [16]:
wv.vectors

array([[-0.25696537, -0.40260807, -0.9845766 , ..., -0.3784373 ,
         0.6256025 ,  0.5775653 ],
       [ 0.7967265 , -1.0895141 ,  0.6663448 , ...,  0.48945996,
         1.2539159 , -1.1740104 ],
       [-1.4117149 , -1.2292888 , -1.231706  , ..., -1.655422  ,
         1.0471157 , -0.5284379 ],
       ...,
       [ 0.12982063,  0.00989456,  0.11360418, ..., -0.11299355,
        -0.02688134,  0.02317555],
       [ 0.02334668, -0.02631488, -0.02490713, ..., -0.00705892,
         0.01536571, -0.0113188 ],
       [ 0.14560641,  0.03414694,  0.12317301, ..., -0.02826422,
        -0.02369511,  0.0310374 ]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `most_similar` function and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [17]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('princess', 0.6244013905525208),
 ('jewel', 0.6065860986709595),
 ('reminiscent', 0.5985398888587952),
 ('queen', 0.5920748114585876),
 ('title', 0.5890806913375854),
 ('symbol', 0.5890547037124634),
 ('diva', 0.5493705868721008),
 ('lover', 0.5451812744140625),
 ('protagonist', 0.5434809923171997),
 ('yellow', 0.5386770963668823)]

As you can see from the output above, your model isn't perfect, but 'Queen' is still in the top 3, and with 'Princess' not too far behind. As you can see from the word in first place, 'reminiscent', your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to created vectorized word embeddings!